## Initialization

In [ ]:
%%capture
# give permissions to kaggle API
! pip install kaggle
! mkdir /root/.kaggle
! cp kaggle.json /root/.kaggle
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
%%capture
# download dataset from kaggle to colab
! kaggle datasets download -d rayonegautam/charanet

# extract the content of the zipped file
!unzip /content/charanet.zip -d /content/

In [ ]:
# remove uncessary files
! rm -r /content/charanet.zip
! rm -r /content/kaggle.json
! rm -r /content/sample_data

## Modules & Libraries

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataset_path = '/content/charaNet'
subfolders = os.listdir(dataset_path)

In [ ]:
categories = {}
label = []
count = []

for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_path, subfolder)
    audio_classes = os.listdir(subfolder_path)

    for audio_class in sorted(audio_classes):
      audio_path = os.path.join(subfolder_path, audio_class)
      audio_list = os.listdir(audio_path)
      print(audio_class)
      count.append(len(audio_list))
      label.append(audio_class)

    break

df = pd.DataFrame({'Label': label, 'Count': count})

Asian Koel
Black Kite
Black-breasted Parrotbill
Black-necked crane
Bristled Grassbird
Cheer Pheasant
Common Cuckoo
Common Pochard
Common Wood Pigeon
Eastern Imperial Eagle
Egyptian Vulture
Great Slaty Woodpecker
Greater Spotted Eagle
Grey Treepie
Grey-crowned Prinia
Grey-sided Thrush
Himalayan Monal
House Crow
House Sparrow
Indian Spotted Eagle
Jerdon_s Babbler
Kashmir Flycatcher
Large-billed Crow
Long-tailed Duck
Pallas_s Fish Eagle
Red-billed Blue Magpie
Rose-ringed Parakeet
Rufous Treepie
Rufous-necked Hornbill
Rustic Bunting
Saker Falcon
Sarus Crane
Satyr Tragopan
Slender-billed Babbler
Spiny Babbler
Spotted Dove
Steppe Eagle
Swamp Francolin
Swamp Grass-babbler
White-throated Bushchat
Wood Snipe
